## Version 2

In [5]:
import time
import datetime  
start_time = time.time()
start_str = datetime.datetime.now().strftime("%Y-%m-%d %H:%M")
print(f"Started: {start_str}")


Started: 2025-09-21 17:16


In [6]:
from collections import defaultdict

# --- Pick a group label from an observation dict ---------------------------
def get_group_label(obs):
    """
    Try common places for a taxon group label.
    Priority:
      1) obs['group'] / obs['taxon_group'] (if you already computed it)
      2) obs['taxon']['iconic_taxon_name'] (iNaturalist)
      3) obs['taxon']['rank'] == 'class' from ancestors (fallback)
      4) 'Other'
    """
    # 1) Precomputed fields
    for k in ("group", "taxon_group"):
        v = obs.get(k)
        if v: 
            return str(v)

    # 2) iNaturalist taxon iconics
    taxon = obs.get("taxon") or {}
    iconic = taxon.get("iconic_taxon_name")
    if iconic:
        return str(iconic)

    # 3) Fallback: try to read 'class' from ancestors if present
    # iNat shape: taxon['ancestors'] is a list of taxon dicts with 'rank'/'name'
    for anc in (taxon.get("ancestors") or []):
        if str(anc.get("rank", "")).lower() == "class" and anc.get("name"):
            return str(anc["name"])

    return "Other"

# --- Build all_groups from a list[dict] or a pandas DataFrame --------------
def build_all_groups(data, group_col=None, whitelist=None, blacklist=None, min_per_group=1):
    """
    data: list of observation dicts OR a pandas DataFrame.
    group_col: if using DataFrame and you already have a group column, name it here.
    whitelist/blacklist: sets of group names to include/exclude.
    min_per_group: groups smaller than this are folded into 'Other'.
    """
    groups = defaultdict(list)

    # Case A: pandas DataFrame
    try:
        import pandas as pd  # only used if df is passed
        is_df = isinstance(data, pd.DataFrame)
    except Exception:
        is_df = False

    if is_df:
        if group_col and group_col in data.columns:
            for g, sub in data.groupby(group_col):
                gname = str(g) if g is not None else "Other"
                if whitelist and gname not in whitelist: 
                    continue
                if blacklist and gname in blacklist: 
                    continue
                groups[gname].extend(sub.to_dict("records"))
        else:
            # derive group from each row
            for _, row in data.iterrows():
                obs = row.to_dict()
                gname = get_group_label(obs)
                if whitelist and gname not in whitelist: 
                    continue
                if blacklist and gname in blacklist: 
                    continue
                groups[gname].append(obs)
    else:
        # Case B: list of dict observations
        for obs in data:
            gname = get_group_label(obs)
            if whitelist and gname not in whitelist: 
                continue
            if blacklist and gname in blacklist: 
                continue
            groups[gname].append(obs)

    # Fold tiny groups into "Other" if requested
    if min_per_group > 1:
        other = []
        for g in list(groups.keys()):
            if len(groups[g]) < min_per_group and g != "Other":
                other.extend(groups.pop(g))
        if other:
            groups["Other"].extend(other)

    # Sort by descending size (nice for reporting/legend order)
    return dict(sorted(groups.items(), key=lambda kv: (-len(kv[1]), kv[0])))

# --- Example usage ---------------------------------------------------------
# 1) From a plain list of observation dicts (e.g., iNat API 'results'):
# observations = results  # list[dict]
# all_groups = build_all_groups(observations, whitelist=None, blacklist=None, min_per_group=3)

# 2) From a pandas DataFrame with a 'group' column you computed earlier:
# all_groups = build_all_groups(df, group_col="group", min_per_group=1)

# Quick sanity print:
# for g, lst in all_groups.items():
#     print(f"{g}: {len(lst)} observations")


In [8]:
# i en notebook-cell (en gång)
!pip install -q tqdm
from tqdm.auto import tqdm 
import json, time, requests, folium
from folium.plugins import TimestampedGeoJson
from shapely.geometry import shape, LineString, MultiLineString, Polygon, MultiPolygon
from shapely.ops import unary_union
from shapely.geometry import mapping

# --- 0) Läs leden från lokal GeoJSON ---
with open("SAT_full.geojson", encoding="utf-8") as f:
    gj = json.load(f)

lines = []
for feat in gj["features"]:
    g = shape(feat["geometry"])
    if isinstance(g, LineString):
        if len(g.coords) >= 2: lines.append(g)
    elif isinstance(g, MultiLineString):
        for seg in g.geoms:
            if len(seg.coords) >= 2: lines.append(seg)

if not lines:
    raise ValueError("SAT_full.geojson innehåller inga linjesegment.")

trail = unary_union(lines)
center_lat, center_lon = trail.centroid.y, trail.centroid.x

# --- 1) Buffert runt leden som sökzon för iNat ---
BUFFER_DEG = 0.003  # ~300 m vid 59–60° lat, justera vid behov
search_geom = trail.buffer(BUFFER_DEG)
if isinstance(search_geom, (Polygon, MultiPolygon)):
    search_geom = search_geom.simplify(0.0002, preserve_topology=True)

# --- 2) iNaturalist helpers (loopar alla delpolygoner) ---  
import time, requests
from shapely.geometry import shape, Polygon, MultiPolygon, LineString
from shapely.ops import unary_union

INAT_URL = "https://api.inaturalist.org/v1/observations"

import time, requests
from shapely.geometry import shape, Polygon, MultiPolygon, LineString
from shapely.ops import unary_union

INAT_URL = "https://api.inaturalist.org/v1/observations"

def _polygon_param_from_poly(poly: Polygon, simplify_tol: float = 0.0002, step: int = 1):
    p = poly.simplify(simplify_tol, preserve_topology=True)
    ring = list(p.exterior.coords)[::step]
    if ring[0] != ring[-1]:
        ring.append(ring[0])
    return ",".join(f"{lng},{lat}" for (lng, lat) in ring)

def _bbox_params_from_poly(poly: Polygon):
    minx, miny, maxx, maxy = poly.bounds
    return {"swlng": minx, "swlat": miny, "nelng": maxx, "nelat": maxy}

def _inat_query_for_polygon(poly: Polygon, params_base: dict, per_page=200, max_pages=5, pause=0.2, pbar=None):
    """
    Kör iNat mot polygon. Har adaptiv förenkling + bbox-fallback.
    Uppdaterar valfri tqdm-progressbar efter varje hämtad sida.
    """
    results = []
    params = dict(params_base)
    params.update({
        "verifiable": "true",
        "order_by": "observed_on",
        "order": "desc",
        "per_page": per_page
    })

    # 1) adaptiv polygon (för att undvika 414)
    for simplify_tol in (0.0002, 0.0004, 0.0008, 0.0016):
        for step in (1, 2, 3, 4):
            polygon_param = _polygon_param_from_poly(poly, simplify_tol=simplify_tol, step=step)
            params_poly = dict(params, polygon=polygon_param)
            try:
                # Första sidan
                r = requests.get(INAT_URL, params=dict(params_poly, page=1), timeout=30)
                if r.status_code == 414:
                    raise requests.HTTPError("414 URI Too Large (polygon)", response=r)
                r.raise_for_status()
                data = r.json()
                batch = data.get("results", [])
                results.extend(batch)
                if pbar: pbar.update(1)

                # Paginerat
                for page in range(2, max_pages + 1):
                    r = requests.get(INAT_URL, params=dict(params_poly, page=page), timeout=30)
                    r.raise_for_status()
                    b2 = r.json().get("results", [])
                    results.extend(b2)
                    if pbar: pbar.update(1)
                    if len(b2) < per_page:
                        break
                    time.sleep(pause)
                return results
            except requests.HTTPError as e:
                # prova nästa simplifiering—bubbla inte upp 414
                if getattr(e, "response", None) is None or e.response.status_code != 414:
                    raise

    # 2) Fallback till bbox
    params_bbox = dict(params, **_bbox_params_from_poly(poly))
    for page in range(1, max_pages + 1):
        r = requests.get(INAT_URL, params=dict(params_bbox, page=page), timeout=30)
        r.raise_for_status()
        b = r.json().get("results", [])
        results.extend(b)
        if pbar: pbar.update(1)
        if len(b) < per_page:
            break
        time.sleep(pause)
    return results

def fetch_inat_all_parts(geom_like,
                         iconic_taxa=None, d1=None, d2=None,
                         per_page=200, max_pages=5, pause=0.2,
                         show_progress=True, desc="Fetch"):
    """
    Tar LineString/Polygon/MultiPolygon, buffrar vid behov, loopar delpolygoner,
    deduplar observationer och visar progressbar.
    """
    g = geom_like if hasattr(geom_like, "geom_type") else shape(geom_like)
    if g.geom_type == "LineString":
        g = g.buffer(0.0015)  # ~150 m
    elif g.geom_type not in ("Polygon", "MultiPolygon"):
        g = unary_union(g)

    polys = [g] if isinstance(g, Polygon) else [p for p in g.geoms if p.area > 1e-10]

    base = {}
    if iconic_taxa: base["iconic_taxa"] = iconic_taxa
    if d1: base["d1"] = d1
    if d2: base["d2"] = d2

    total_steps = len(polys) * max_pages  # ungefärligt (per polygon x sidor)
    pbar = tqdm(total=total_steps, desc=desc, unit="page", leave=False) if show_progress else None

    all_obs = {}
    try:
        for poly in polys:
            part = _inat_query_for_polygon(poly, base, per_page=per_page, max_pages=max_pages, pause=pause, pbar=pbar)
            for o in part:
                all_obs[o["id"]] = o
    finally:
        if pbar: pbar.close()
    return list(all_obs.values())


# --- 3) Hämta data (~1000 per kategori om det finns) --- 
from datetime import date, timedelta
d2 = date.today().isoformat()
d1 = (date.today() - timedelta(days=20)).isoformat()  # senaste 30 dagarna

# Växter
plants = fetch_inat_all_parts(search_geom, iconic_taxa="Plantae", d1=d1, d2=d2,
                              per_page=200, max_pages=5, show_progress=True, desc="Plantae")

# Djur (visa övergripande progress för taxagrupper)
animal_taxa = ["Aves","Mammalia","Reptilia","Amphibia","Insecta","Arachnida","Mollusca","Animalia"]
animals_all = []
with tqdm(total=len(animal_taxa), desc="Animal taxa", unit="taxon") as t_taxa:
    for tx in animal_taxa:
        part = fetch_inat_all_parts(search_geom, iconic_taxa=tx, d1=d1, d2=d2,
                                    per_page=200, max_pages=5, show_progress=True, desc=tx)
        animals_all.extend(part)
        t_taxa.update(1)
animals = list({o["id"]: o for o in animals_all}.values())
print(f"Växter: {len(plants)} | Djur: {len(animals)}")

# --- 4) Bygg FeatureCollections med 'time' + bild i popup ---
def obs_to_feature(o):
    if not o.get("geojson"): return None
    lon, lat = o["geojson"]["coordinates"]
    date = o.get("observed_on") or o.get("created_at") or ""
    taxon = o.get("taxon") or {}
    cname = taxon.get("preferred_common_name") or taxon.get("name") or "okänd"
    sname = taxon.get("name") or ""
    url   = f"https://www.inaturalist.org/observations/{o['id']}"
    img = ""
    photos = o.get("photos") or []
    if photos:
        raw = photos[0].get("url", "")
        if raw: img = raw.replace("square","medium")

    popup = f"<b>{cname}</b><br><i>{sname}</i><br>{date}<br>"
    if img: popup += f"<img src='{img}' width='220'><br>"
    popup += f"<a href='{url}' target='_blank'>Visa i iNaturalist</a>"

    return {
        "type":"Feature",
        "geometry":{"type":"Point","coordinates":[lon,lat]},
        "properties":{"time":date,"popup":popup}
    }

def to_fc(obs): 
    feats = [obs_to_feature(o) for o in obs]
    return {"type":"FeatureCollection","features":[f for f in feats if f]}

plants_fc  = to_fc(plants)
animals_fc = to_fc(animals)

# --- 5) Karta med separata tidslager ---
m = folium.Map(location=[center_lat, center_lon], zoom_start=10, control_scale=True)

# leden
folium.GeoJson(mapping(trail), name="SAT-led",
               style_function=lambda x: {"color":"#1e40af","weight":3}).add_to(m)

# (valfritt) visa bufferten
if isinstance(search_geom,(Polygon,MultiPolygon)):
    folium.GeoJson(mapping(search_geom), name="Sökzon",
                   style_function=lambda x: {"color":"#3388ff","weight":1,"fillOpacity":0.05}).add_to(m)

TimestampedGeoJson(plants_fc,  period="P1D", duration="P3D",
                   add_last_point=True, auto_play=True,  loop=True,
                   date_options="YYYY-MM-DD", name="Växter – tidsserie").add_to(m)

TimestampedGeoJson(animals_fc, period="P1D", duration="P3D",
                   add_last_point=True, auto_play=False, loop=True,
                   date_options="YYYY-MM-DD", name="Djur – tidsserie").add_to(m)

folium.LayerControl(collapsed=False).add_to(m)
m.save("inat_realtime_poc2.html")
print("Sparade: inat_realtime_poc2.html")


Plantae:   0%|          | 0/80 [00:00<?, ?page/s]

Animal taxa:   0%|          | 0/8 [00:00<?, ?taxon/s]

Aves:   0%|          | 0/80 [00:00<?, ?page/s]

Mammalia:   0%|          | 0/80 [00:00<?, ?page/s]

Reptilia:   0%|          | 0/80 [00:00<?, ?page/s]

...

Arachnida:   0%|          | 0/80 [00:00<?, ?page/s]

Mollusca:   0%|          | 0/80 [00:00<?, ?page/s]

Animalia:   0%|          | 0/80 [00:00<?, ?page/s]

Växter: 1097 | Djur: 8121


TypeError: TimestampedGeoJson.__init__() got an unexpected keyword argument 'name'

In [9]:
from shapely.geometry import mapping
import folium
from folium.plugins import TimestampedGeoJson

# --- Färgkodning per taxa ---
TAXON_COLORS = {
    "Plantae":   "#2b8a3e",  # grön
    "Aves":      "#1d4ed8",  # blå
    "Mammalia":  "#78350f",  # brun
    "Reptilia":  "#059669",  # grön-turkos
    "Amphibia":  "#14b8a6",  # cyan
    "Insecta":   "#eab308",  # gul
    "Arachnida": "#7f1d1d",  # mörkröd
    "Mollusca":  "#9333ea",  # lila
    "Animalia":  "#6b7280",  # grå
}

# --- Helper: observation → Feature med popup ---
def obs_to_feature(o, color="#3388ff"):
    if not o.get("geojson"):
        return None
    lon, lat = o["geojson"]["coordinates"]
    date = o.get("observed_on") or o.get("created_at") or ""
    taxon = o.get("taxon") or {}
    cname = taxon.get("preferred_common_name") or taxon.get("name") or "okänd"
    sname = taxon.get("name") or ""
    url   = f"https://www.inaturalist.org/observations/{o['id']}"

    img = ""
    photos = o.get("photos") or []
    if photos:
        raw = photos[0].get("url", "")
        if raw:
            img = raw.replace("square","medium")

    popup = f"<b>{cname}</b><br><i>{sname}</i><br>{date}<br>"
    if img:
        popup += f"<img src='{img}' width='220'><br>"
    popup += f"<a href='{url}' target='_blank'>Visa i iNaturalist</a>"

    return {
        "type": "Feature",
        "geometry": {"type": "Point", "coordinates": [lon, lat]},
        "properties": {
            "time": date,
            "popup": popup,
            "icon": "circle",
            "iconstyle": {
                "color": color,
                "fillColor": color,
                "fillOpacity": 0.8
            }
        }
    }

def to_fc(obs, color="#3388ff"): 
    feats = [obs_to_feature(o, color) for o in obs]
    return {"type": "FeatureCollection", "features": [f for f in feats if f]}

# --- Taxagrupper att hämta ---
animal_taxa = ["Aves","Mammalia","Reptilia","Amphibia","Insecta","Arachnida","Mollusca","Animalia"]
all_groups = {"Plantae": plants}
for tx in animal_taxa:
    obs = fetch_inat_all_parts(search_geom, iconic_taxa=tx, d1=d1, d2=d2,
                               per_page=200, max_pages=5, show_progress=True, desc=tx)
    all_groups[tx] = obs

# --- Kartan ---
m = folium.Map(location=[center_lat, center_lon], zoom_start=10, control_scale=True)

# SAT-leden
folium.GeoJson(
    mapping(trail),
    name="SAT-led",
    style_function=lambda x: {"color": "#1e40af", "weight": 3}
).add_to(m)

# (valfritt) Bufferten
folium.GeoJson(
    mapping(search_geom),
    name="Sökzon",
    style_function=lambda x: {"color": "#3388ff","weight":1,"fillOpacity":0.05}
).add_to(m)

# --- Lägg till ett tidslager per taxa med egen färg ---
for tx, obs in all_groups.items():
    color = TAXON_COLORS.get(tx, "#3388ff")
    fc = to_fc(obs, color=color)
    fg = folium.FeatureGroup(name=f"{tx} – tidsserie")
    TimestampedGeoJson(
        fc,
        period="P1D",
        duration="P3D",
        add_last_point=True,
        auto_play=False,
        loop=True,
        max_speed=1,
        loop_button=True,
        date_options="YYYY-MM-DD",
        time_slider_drag_update=True
    ).add_to(fg)
    fg.add_to(m)

folium.LayerControl(collapsed=False).add_to(m)
m.save("inat_taxa_layers_colored.html")
print("✅ Sparade: inat_taxa_layers_colored.html")


Aves:   0%|          | 0/80 [00:00<?, ?page/s]

Mammalia:   0%|          | 0/80 [00:00<?, ?page/s]

Reptilia:   0%|          | 0/80 [00:00<?, ?page/s]

Amphibia:   0%|          | 0/80 [00:00<?, ?page/s]

Arachnida:   0%|          | 0/80 [00:00<?, ?page/s]

Mollusca:   0%|          | 0/80 [00:00<?, ?page/s]

Animalia:   0%|          | 0/80 [00:00<?, ?page/s]

AssertionError: TimestampedGeoJson can only be added to a Map object.

In [10]:
# --- Lägg till ett tidslager per taxa med egen färg ---
all_bounds = []
for tx, obs in tqdm(all_groups.items(), desc="Bygger tidslager", total=len(all_groups)):
    color = TAXON_COLORS.get(tx, "#3388ff")
    fc, bounds, skipped = to_fc(obs, color=color, desc=f"{tx} → features")
    all_bounds.extend(bounds)

    # Folium versions differ: some don't support name/control on TimestampedGeoJson
    try:
        layer = TimestampedGeoJson(
            fc,
            period="P1D",
            duration="P3D",
            add_last_point=True,
            auto_play=False,
            loop=True,
            max_speed=1,
            loop_button=True,
            date_options="YYYY-MM-DD",
            time_slider_drag_update=True,
            name=f"{tx} – tidsserie",   # may raise TypeError on older versions
            control=True                 # may raise TypeError on older versions
        )
    except TypeError:
        layer = TimestampedGeoJson(
            fc,
            period="P1D",
            duration="P3D",
            add_last_point=True,
            auto_play=False,
            loop=True,
            max_speed=1,
            loop_button=True,
            date_options="YYYY-MM-DD",
            time_slider_drag_update=True
        )

    layer.add_to(m)

    if skipped:
        tqdm.write(f"⚠️ {tx}: hoppade över {skipped} obs utan koordinater/bild/format.")

# Auto-zoom om vi har punkter
if all_bounds:
    m.fit_bounds(all_bounds)

folium.LayerControl(collapsed=False).add_to(m)  # OBS: visar bara vanliga lager, inte tidslagren i äldre Folium
m.save("inat_taxa_layers_colored_2.html")
print("✅ Sparade: inat_taxa_layers_colored_2.html")


Bygger tidslager:   0%|          | 0/9 [00:00<?, ?it/s]

TypeError: to_fc() got an unexpected keyword argument 'desc'

In [ ]:
    end_time = time.time()
    duration = end_time - start_time
    print(f"GeoJSON saved to {output_path}")
    print(f"Finished in {duration:.2f} seconds.")
